In [1]:
import numpy as np

In [2]:
file_path = "./ddi_with_type_latest.txt"
with open(file_path, "r") as f:
    readfile = f.read()

In [3]:
reflection_list = []
vector_set = set()
max_vector = 0
for item in readfile.split(sep="\n"):
    if len(item.split(sep="\t")) == 3:
        single_item = item.split(sep="\t")
        reflection_list.append(single_item)
        vector_set.add(single_item[1])
        vector_set.add(single_item[0])
        max_vector = max(max_vector, max(int(single_item[0]), int(single_item[1])))

## Q1 网络中节点和边的数量。

In [4]:
# number of nodes
len(vector_set)

1603

In [5]:
# number of edges
len(reflection_list)

38405

## Q2 找出度最大的前20个节点，并计算所有节点的平均度。

In [6]:
# Use dict to restore the degree of every nodes
node_degree = {str(i):0 for i in range(len(vector_set))}
for reflect in reflection_list:
    node_degree[reflect[0]] += 1
    node_degree[reflect[1]] += 1

In [7]:
# Order nodes by sum_degree and cut the top20
a = sorted(node_degree.items(), key=lambda x: x[1], reverse=True)
a[:20]

[('848', 194),
 ('350', 188),
 ('870', 186),
 ('118', 181),
 ('83', 177),
 ('972', 176),
 ('88', 174),
 ('1440', 173),
 ('1339', 166),
 ('446', 165),
 ('63', 163),
 ('140', 163),
 ('22', 162),
 ('187', 160),
 ('697', 159),
 ('441', 158),
 ('811', 157),
 ('33', 156),
 ('169', 155),
 ('266', 154)]

In [8]:
# Calculate the average degree of all nodes
average_degree = sum(node_degree.values())/len(vector_set)
average_degree

47.91640673736744

## Q3 统计网络中共有多少种不同类型的边，类型为47的边有多少条

In [9]:
# Count the nums of types
# Weight is equal to the index of type!!!
types = set()
cnt_47 = 0
for reflect in reflection_list:
    types.add(reflect[2])
    if reflect[2] == '47':
        cnt_47 += 1

In [10]:
# Total nums of different edges
len(types)

86

In [11]:
# Nums of edges with weight "47"
cnt_47

6864

## Q4 统计节点度分布,画出度分布图。

## Q5 找出给定节点i和j之间的所有最短路径，并输出。

#### （8,309）、（67,850）、（990,1256）

In [12]:
# Dijkstra algorithm
M = 1e20

def generate_matrix(node_cnt, array=[]) -> list:
    # Initialize node_cnt * node_cnt Matrix
    matrix = [[M for k in range(int(node_cnt))] for i in range(int(node_cnt))]
    for i in array:
        matrix[i[0]][i[1]] = 1
    return matrix
 
def min_index(arr=[], indexs=[]) -> int:
    # Calculate the node ordinal of the shortest path in a given node
    min_value = arr[0]
    index = indexs[0]
    for i in indexs:
        if arr[i] < min_value:
            min_value = arr[i]
            index = i
    return index
 
def dijkstra(matrix, origin) -> tuple:
    # Use Dijkstra algorithm to find the shortest path in a directed graph
    assert 0 <= origin < len(matrix) 
    s = [origin]
    dist = matrix[origin]
    vs = [i for i in range(len(dist))]
    vs.remove(origin)
    
    p = [-1 if i == M else origin for i in dist]
 
    while len(vs) > 0:
        
        node = min_index(arr=dist, indexs=vs)
        s.append(node)
        vs.remove(node)
        for i in vs:
            if matrix[node][i] != M:
                if dist[node] + matrix[node][i] < dist[i]:
                    dist[i] = dist[node] + matrix[node][i]
                    p[i] = node
    
    dist[origin] = 0
    return dist, p
 
def route(p=[]) -> list:
    i = 0
    result = []
    while i < len(p):
        m = len(p) - 1 - i
        path = [m]
        while p[m] != -1:
            path.append(p[m])
            m = p[m]
        path.reverse()
        result.append((len(p) - 1 - i, path))
        i += 1
    return result
 
if __name__ == '__main__':
    node_cnt = len(vector_set)
    array = []
    for i in reflection_list:
        array.append((int(i[0]), int(i[1])))
    matrix = generate_matrix(node_cnt=node_cnt, array=array)
    start = int(input(u'请输入起始位置（0 ~ %d）：' % (int(node_cnt - 1))))
    dist, p = dijkstra(matrix, start)
    target = int(input(u'请输入终点位置（0 ~ %d）：' % (int(node_cnt - 1))))
    for item in route(p):
        if item[0] == target:
            if dist[item[0]] != M:
                print("%d 点到 %d 点的最短距离为 %d，走过的路径为：%s" % (start, item[0], dist[item[0]], item[1]))
            else:
                print("%d 点到 %d 点不可达" % (start, item[0]))

请输入起始位置（0 ~ 1602）：67
请输入终点位置（0 ~ 1602）：850
67 点到 850 点的最短距离为 3，走过的路径为：[67, 87, 104, 850]


## Q6 求所有节点的聚集系数以及平均聚集系数

In [13]:
# Ei = sum（edge(node2node)）bewteen every node connected with current node i
# Ci = (2Ei)/(ki(ki-1))

In [17]:
cites = {}
eis = {}

for item in vector_set:
    edges_node = []
    ei = 0
    for reflect in reflection_list:
        if item == reflect[0]:
            edges_node.append(reflect[1])
        elif item == reflect[1]:
            edges_node.append(reflect[0])
    cites[item] = edges_node
    
    for node in edges_node:
        for reflect in reflection_list:
            if node == reflect[0] and reflect[1] in edges_node:
                ei += 1
    eis[item] = ei

# eis

In [18]:
# Calculate clustering coefficient with eis base on formula given above
clus_coefficient = {}
for item in eis:
    if len(cites[item])>1:
        clus_coefficient[item] = (2*eis[item])/(len(cites[item])*(len(cites[item])-1))
    else:
        clus_coefficient[item] = 0.0
clus_coefficient

{'1025': 0.039886039886039885,
 '608': 0.11851851851851852,
 '433': 0.16666666666666666,
 '1564': 0.18181818181818182,
 '1357': 0.16666666666666666,
 '1378': 0.0,
 '806': 0.054945054945054944,
 '308': 0.13250517598343686,
 '894': 0.05957767722473605,
 '903': 0.07560975609756097,
 '1482': 0.0,
 '254': 0.2129032258064516,
 '72': 0.2,
 '636': 0.14366998577524892,
 '1090': 0.12,
 '1132': 0.1694915254237288,
 '643': 0.12897959183673469,
 '1573': 0.20346320346320346,
 '455': 0.10451152223304122,
 '932': 0.0803233574131407,
 '619': 0.13953488372093023,
 '173': 0.1455026455026455,
 '204': 0.07076923076923076,
 '734': 0.08158508158508158,
 '85': 0.0890909090909091,
 '1497': 0.13732004429678848,
 '1008': 0.12131147540983607,
 '283': 0.0,
 '875': 0.16205533596837945,
 '1162': 0.16378269617706237,
 '1530': 0.0,
 '609': 0.10443307757885763,
 '812': 0.15555555555555556,
 '995': 0.07700851536467974,
 '1214': 0.07276887871853546,
 '862': 0.125544267053701,
 '201': 0.16666666666666666,
 '422': 0.055555

In [19]:
average_clus_coefficient = sum(clus_coefficient.values()) / len(clus_coefficient)
average_clus_coefficient

0.10799533208547336

## Q7 求图中连通分量的个数

<img src="./Ques_img/Q7.png" width = "480" height = "480"/> 

In [41]:
# For new graph
reflection_graph = {'A': set(['B', 'C', 'D']),
                    'B': set(['A', 'C']),
                    'C': set(['A', 'B']),
                    'D': set(['A']),
                    'F': set(['G']),
                    'G': set(['F']),
                    'H': set([])}

In [58]:
close_list = []
res = 0

def stack(i,graph,close_list):
    stack = []
    stack.append(i)
    while stack:
        i = stack.pop(-1)
        for t in graph[i]:
            if t not in close_list:
                close_list.append(t)
                stack.append(t)
        
for node in reflection_graph.keys():
    if node not in close_list:
        print(f"{node} Create new component")
        close_list.append(node)
        res += 1
        stack(node,reflection_graph,close_list)
    else:
        print(f"{node} Already done")

print(f"连通分量的数量为 {res}")

A Create new component
B Already done
C Already done
D Already done
F Create new component
G Already done
H Create new component
连通分量的数量为 3


## Q8 求图中满足要求的子图（induced subgraph）个数

In [60]:
    <img src="./Ques_img/Q8.png" width = "480" height = "480"/> 

SyntaxError: invalid syntax (730407019.py, line 1)